In [10]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import os
import numpy as np
import shutil

## check if page downloaded successfully

In [11]:
url = "https://www.dli.pa.gov/Individuals/Workforce-Development/warn/notices/Pages/default.aspx"
page = requests.get(url)
page.status_code

200

## webpage url parse

In [12]:
soup = BeautifulSoup(page.text, 'html.parser')
data = soup.findAll('div',attrs={'class':'ms-rtestate-field'})
month_links = []
for div in data:
    links = div.findAll('a')
    for a in links:
        month_links.append("https://www.dli.pa.gov/"+a['href'])

In [13]:
len(month_links)

111

In [14]:
#month_links

## after 2016

In [15]:
 def get_pattern_unbold(element):
    ele = element
    data = ele.find_all('p')
    if data:
        try:
            name = re.sub(' +', ' ', " ".join([data[0].find('strong').text.replace('\n', ' ')])).strip() 
        except: 
            return None
        if len(data) != 2:
            return name

        ad_list = []
        #delete the name part
        ad_list.append(", ".join([ele.strip() for ele in re.findall(r'\>([^\>]*)\<', str(data[0]).split("</strong>")[1]) if ele not in ('\n','')]))

        for ad in data[1:-1]:

            ad_list.append(", ".join([ele.strip() for ele in re.findall(r'\>([^\>]*)\<', str(ad)) if ele not in ('\n','')]))

        part1 = " and ".join(ad_list)

        #print(part1)

        part2 = " ".join([ele.strip() for ele in re.findall(r'\>([^\>]*)\<', str(data[-1])) if ele not in ('\n','')])

        string = part1+" "+part2
        
        string = re.sub(' +', ' ', " ".join([string.replace('\n', ' ')])).strip()

        try:
            address = string.split("COUNTY:")[0]

            county = string.split("COUNTY:")[1].split("# AFFECTED:")[0]

            string = string.replace("LAYOFF EFFECTIVE DATES:", "EFFECTIVE DATE:")

            string = string.replace("LAYOFF EFFECTIVE DATE:", "EFFECTIVE DATE:")

            #print(string.split("COUNTY:")[1].split("# AFFECTED:"))

            employees_affected = string.split("COUNTY:")[1].split("# AFFECTED:")[1].split("EFFECTIVE DATE:")[0]

            string = string.replace("CLOSING OR LAYOFF:", "CLOSURE OR LAYOFF:")

            effective_date = string.split("COUNTY:")[1].split("# AFFECTED:")[1].split("EFFECTIVE DATE:")[1].split("CLOSURE OR LAYOFF:")[0]



            closure_or_layoff = string.split("COUNTY:")[1].split("# AFFECTED:")[1].split("EFFECTIVE DATE:")[1].split("CLOSURE OR LAYOFF:")[1]

            return name, address, county, employees_affected, effective_date, closure_or_layoff
        except:
            return None



## 2016

In [16]:
def get_pattern_bold(element):
    data = element
    if data:
        total_text = re.sub(' +', ' ', " ".join([data.text.replace('\n', ' ')])).strip()
        bold_ele = [re.sub(' +', ' ', ele.text.replace('\n', ' ')).strip() for ele in data.find_all('strong')]
        if bold_ele == []:
            return None

        name =  bold_ele[0].strip()
        #if "*UPDATE*" in name:
            #return " ".join(bold_ele)
        
        for ele in bold_ele:
            total_text = total_text.replace(ele, "@#$%&")
        results = [re.sub(' +', ' ', ele).strip() for ele in total_text.split("@#$%&")]
        #remove null elements
        results = [ele for ele in results if ele]
        #print(bold_ele)
        #print(results)
        if len(bold_ele) == 4 and len(results) == 4:
            #print(bold_ele)
            address = results[0]
            county = results[1]
            employees_affected = results[2]
            effective_date = results[3]
            closure_or_layoff = "N/A"
            return name, address, county, employees_affected, effective_date, closure_or_layoff
        elif len(bold_ele) == 5 and len(results) == 4:# and bold_ele[3] in ['CLOSURE',"CLOSING", "LAYOFF","PERMANENT CLOSURE", "PERMANENT LAYOFF","PERMANENT CLOSING", "CONTRACT CANCELLED","PLANT CLOSURE","CONTRACT CANCELLED","PLANT CLOSURE","CLOSURE/PERMANENT LAYOFF","CLOSURE/PENDING SALE","PERMANENT LAYOFFS"， "PENDING SALE - CLOSING"，"PLANT CLOSING", "PENDING SALE - NO LAYOFF", "PERMANENT", "PERMANENT MASS LAYOFF" ]:
            address = results[0]
            county = results[1]
            employees_affected = results[2]
            effective_date = results[3]
            closure_or_layoff = bold_ele[3]
            return name, address, county, employees_affected, effective_date, closure_or_layoff
        else:
            #print(bold_ele)
            #print(results)
            #print()
            return " ".join(bold_ele[:2])

        if name in company_name and effective_date in effective_date_l and address in company_address:
            return None
        


In [17]:
def get_df(url):
    #
    page = requests.get(url)
    #store html
    url_name = url.split('/')[-1].split('.')[0]
    #get year
    year = int(url_name.split('-')[1])
    with open(os.path.join(dirs, url_name+".html"), 'w',encoding = 'utf8') as file:
        file.write(page.text)
    soup = BeautifulSoup(page.text, 'html.parser')
    table_data = soup.findAll('td')
    table_data = [ele for ele in table_data if re.sub(' +', ' ', ele.text).strip()]
    #
    urls = []
    company_name = []
    company_address = []
    county_l = []
    employees_affected_l = []
    effective_date_l = []
    closure_or_layoff_l = []
    
    #
    error_link = []
    error_name = []

    for ele in table_data:#[10:11]:
        if year < 2016:
            result = get_pattern_bold(ele)
            if result == None:
                continue
            if len(result) == 6:

                name, address, county, employees_affected, effective_date, closure_or_layoff = result

                if name in company_name and effective_date in effective_date_l and address in company_address and county in county_l and employees_affected in employees_affected_l and closure_or_layoff in closure_or_layoff_l:
                    continue
                
                urls.append(url) 
                
                company_name.append(name)

                company_address.append(address)

                county_l.append(county)

                employees_affected_l.append(employees_affected)

                effective_date_l.append(effective_date)

                closure_or_layoff_l.append(closure_or_layoff)

            else:
                name = result 
                error_name.append(name)
                error_link.append(url)
                continue
                
        if year > 2016:
            result = get_pattern_unbold(ele)
            
            if result == None:
                continue
            if len(result) == 6 :

                name, address, county, employees_affected, effective_date, closure_or_layoff = result

                if name in company_name and effective_date in effective_date_l and address in company_address and county in county_l and employees_affected in employees_affected_l and closure_or_layoff in closure_or_layoff_l:
                    continue
                if not address:
                    name = name
                    error_name.append(name)
                    error_link.append(url)
                    continue                    
                urls.append(url) 
                    
                company_name.append(name)

                company_address.append(address)

                county_l.append(county)

                employees_affected_l.append(employees_affected)

                effective_date_l.append(effective_date)

                closure_or_layoff_l.append(closure_or_layoff)

            else:
                name = result 
                error_name.append(name)
                error_link.append(url)
                continue
    #create dataframe
    d1 = {'url':urls, 'company_name': company_name, 'company_address': company_address, 'county': county_l, 'employees_affected': employees_affected_l, 'effective_date': effective_date_l, 'closure_or_layoff': closure_or_layoff_l}
    df1 = pd.DataFrame(data=d1)
    
    #create error dataframe
    d2 = {'links': error_link, 'name': error_name}
    df2 = pd.DataFrame(data=d2)
    df2 = df2.drop_duplicates()
    print(url.split('/')[-1].split('.')[0]+' is done!')
    return df1, df2



In [18]:

dirs = "./html"
if os.path.exists(dirs):
    shutil.rmtree(dirs)
os.makedirs(dirs)
result_dirs = "./results"
if os.path.exists(result_dirs):
    shutil.rmtree(result_dirs)
os.makedirs(result_dirs)
df_lists = []
df_error_lists = []
for ele in month_links:#[98:110]:
    #result = get_df(ele)
    date = ele.split('/')[-1].split('.')[0]
    #try:
    result = get_df(ele)
    sub_result_dirs = "./results/"+date+"/"
    if os.path.exists(sub_result_dirs):
        shutil.rmtree(sub_result_dirs)
    os.makedirs(sub_result_dirs)
    result[0].to_csv("./results/"+date+"/result.csv", index = False)
    result[1].to_csv("./results/"+date+"/error.csv", index = False)
    df_lists.append(result[0])
    df_error_lists.append(result[1])
    #except:
        #print(ele)
df1 = pd.concat(df_lists)
df1.to_csv("final_result.csv", index = False)
df2 = pd.concat(df_error_lists)
df2.to_csv("final_error.csv", index = False)

March-2020 is done!
February-2020 is done!
January-2020 is done!
December-2019 is done!
November-2019 is done!
October-2019 is done!
September-2019 is done!
August-2019 is done!
July-2019 is done!
June-2019 is done!
May-2019 is done!
April-2019 is done!
March-2019 is done!
February-2019 is done!
January-2019 is done!
December-2018 is done!
November-2018 is done!
October-2018 is done!
September-2018 is done!
August-2018 is done!
July-2018 is done!
June-2018 is done!
May-2018 is done!
April-2018 is done!
March-2018 is done!
February-2018 is done!
January-2018 is done!
December-2017 is done!
November-2017 is done!
October-2017 is done!
September-2017 is done!
August-2017 is done!
July-2017 is done!
June-2017 is done!
May-2017 is done!
April-2017 is done!
March-2017 is done!
February-2017 is done!
January-2017 is done!
December-2016 is done!
November-2016 is done!
October-2016 is done!
September-2016 is done!
August-2016 is done!
July-2016 is done!
June-2016 is done!
May-2016 is done!
Apri